In [ ]:
%load_ext autoreload

import pybamm
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from pybamm_bayes.workflows.utils.parameter_sets.airbus_nmc import airbus_cell_parameters

pybamm.set_logging_level("SUCCESS")
data = pd.read_csv("scratch/SN01_cycle_1.csv")

In [ ]:
%autoreload 2
parameter_values = airbus_cell_parameters()
param = pybamm.LithiumIonParameters()

Vinit = data.output.values[0]
eps_n = 0.59658823
eps_p = 0.79010106
Q_Li = 3.94666883

# sample
parameter_values["Negative electrode active material volume fraction"] = eps_n
parameter_values["Positive electrode active material volume fraction"] = eps_p
Q_n = parameter_values.evaluate(param.n.Q_init)
Q_p = parameter_values.evaluate(param.p.Q_init)
U_n = param.n.prim.U
U_p = param.p.prim.U
T_ref = param.T_ref

In [ ]:
model = pybamm.BaseModel()

Vmin = data.output.min()
Vmax = data.output.max()

# equation for 100% SOC
x_100 = pybamm.Variable("x_100")
y_100 = (Q_Li - x_100 * Q_n) / Q_p

# equation for 0% SOC
x_0 = pybamm.Variable("x_0")
Q = Q_n * (x_100 - x_0)
y_0 = y_100 + Q / Q_p

# equation for SOC
soc = pybamm.Variable("SOC")
x = x_0 + soc * (x_100 - x_0)
y = y_0 - soc * (y_0 - y_100)

model.algebraic = {
    x_100: U_p(y_100, T_ref) - U_n(x_100, T_ref) - Vmax,
    x_0: U_p(y_0, T_ref) - U_n(x_0, T_ref) - Vmin,
    soc: U_p(y, T_ref) - U_n(x, T_ref) - Vinit
    }

model.initial_conditions = {x_100: 0.995, x_0: 0.02, soc: 0.9}

model.variables = {
    "x": x, 
    "y": y,
    "SOC": soc
    }

sim = pybamm.Simulation(model, parameter_values=parameter_values)
sol = sim.solve([0])

x = sol["x"].data[0]
y = sol["y"].data[0]
soc = sol["SOC"].data[0]

In [ ]:
data["time"] = data["time"].values - data["time"].values[0]

parameter_values["Maximum concentration in negative electrode [mol.m-3]"] = 41360
current = pybamm.Interpolant(x = data.time.values, y = -1*data.current.values, children=pybamm.t)
model = pybamm.lithium_ion.SPMe()
parameter_values = airbus_cell_parameters()

parameter_values["Negative electrode active material volume fraction"] = eps_n
c_n_max = parameter_values["Maximum concentration in negative electrode [mol.m-3]"]
parameter_values["Initial concentration in negative electrode [mol.m-3]"] = c_n_max * x

parameter_values["Positive electrode active material volume fraction"] = eps_p
c_p_max = parameter_values["Maximum concentration in positive electrode [mol.m-3]"]
parameter_values["Initial concentration in positive electrode [mol.m-3]"] = c_p_max * y


parameter_values["Current function [A]"] = current

simulation = pybamm.Simulation(
    model,
    solver=pybamm.CasadiSolver("fast"),
    parameter_values=parameter_values,
)



In [ ]:
sol = simulation.solve(initial_soc=soc)

In [ ]:
plt.clf()
sol.plot()

In [ ]:
fig, ax = pybamm.plot_voltage_components(sol, split_by_electrode=True)
ax.set_ylim(2, 5) 
fig

In [ ]:
plt.clf()
sol.plot(
    [
        "Terminal voltage [V]",
        "Negative particle concentration overpotential [V]",
        "X-averaged negative particle concentration",
        "Negative particle concentration",
        "Maximum negative particle concentration [mol.m-3]"
    ]
)